In [43]:
from neo4j import GraphDatabase, basic_auth
from paths import *
import pandas as pd
import os


## Mapping short chain fatty acids to SPOKE nodes

In [82]:
short_chain_spoke_map = [
    {
        "name" : "2-Methylbutyric acid",
        "identifier": "inchikey:WLAMNBDJUVNPJU-UHFFFAOYSA-N"
    },
    {
        "name" : "Acetic acid",
        "identifier" : "inchikey:QTBSBXVTEAMEQO-UHFFFAOYSA-N"
    },
    {
        "name" : "Butyric acid",
        "identifier" : "inchikey:FERIUCNNQQJTOY-UHFFFAOYSA-N"
    },
    {
        "name" : "Hexanoic acid",
        "identifier" : "inchikey:FUZZWVXGSFPDMH-UHFFFAOYSA-N"
    },
    {
        "name" : "Isobutyric acid",
        "identifier" : "inchikey:KQNPFQTWMSNSAP-UHFFFAOYSA-N"
    },
    {
        "name" : "Isovaleric acid",
        "identifier" : "inchikey:GWYFCOCPABKNJV-UHFFFAOYSA-N"
    },
    {
        "name" : "Propionic acid",
        "identifier": "inchikey:XBDQKXXYIPTUBI-UHFFFAOYSA-N"
    },
    {
        "name" : "Valeric acid",
        "identifier" : "inchikey:NQPDZGIKBAWPEJ-UHFFFAOYSA-N"
    }    
]
 
short_chain_spoke_map_df = pd.DataFrame(short_chain_spoke_map)
short_chain_spoke_map_df = short_chain_spoke_map_df.rename(columns={"identifier":"spoke_identifer"})
short_chain_spoke_map_df

,name,spoke_identifer
0,2-Methylbutyric acid,inchikey:WLAMNBDJUVNPJU-UHFFFAOYSA-N
1,Acetic acid,inchikey:QTBSBXVTEAMEQO-UHFFFAOYSA-N
2,Butyric acid,inchikey:FERIUCNNQQJTOY-UHFFFAOYSA-N
3,Hexanoic acid,inchikey:FUZZWVXGSFPDMH-UHFFFAOYSA-N
4,Isobutyric acid,inchikey:KQNPFQTWMSNSAP-UHFFFAOYSA-N
5,Isovaleric acid,inchikey:GWYFCOCPABKNJV-UHFFFAOYSA-N
6,Propionic acid,inchikey:XBDQKXXYIPTUBI-UHFFFAOYSA-N
7,Valeric acid,inchikey:NQPDZGIKBAWPEJ-UHFFFAOYSA-N


## Comparing names of short chain fatty acids from iMSMS table and SPOKE

In [83]:
query = "MATCH(n:Compound) WHERE n.identifier = '{}' RETURN n.name as n_name"

auth = basic_auth(SPOKE_USER, SPOKE_PASSWORD)
sdb = GraphDatabase.driver(URI, auth=auth)
node_list = []
with sdb.session() as session:
    with session.begin_transaction() as tx:
        for index,row in short_chain_spoke_map_df.iterrows():
            result = tx.run(query.format(row["spoke_identifer"]))
            for row_ in result:
                node_list.append((row["spoke_identifer"], row["name"], row_["n_name"]))
sdb.close()
node_df = pd.DataFrame(node_list, columns = ["spoke_identifer", "metabolon_name", "spoke_name"])
node_df

,spoke_identifer,metabolon_name,spoke_name
0,inchikey:WLAMNBDJUVNPJU-UHFFFAOYSA-N,2-Methylbutyric acid,2-Methylbutanoic acid
1,inchikey:QTBSBXVTEAMEQO-UHFFFAOYSA-N,Acetic acid,Acetic Acid
2,inchikey:FERIUCNNQQJTOY-UHFFFAOYSA-N,Butyric acid,Butyric Acid
3,inchikey:FUZZWVXGSFPDMH-UHFFFAOYSA-N,Hexanoic acid,Hexanoic acid
4,inchikey:KQNPFQTWMSNSAP-UHFFFAOYSA-N,Isobutyric acid,Isobutyric acid
5,inchikey:GWYFCOCPABKNJV-UHFFFAOYSA-N,Isovaleric acid,Isovaleric acid
6,inchikey:XBDQKXXYIPTUBI-UHFFFAOYSA-N,Propionic acid,Propionic Acid
7,inchikey:NQPDZGIKBAWPEJ-UHFFFAOYSA-N,Valeric acid,Valeric acid


## Mapping Compounds in the Global metabolomics files

In [ ]:
filename = GLOBAL_SERUM_DATA_FILENAME

file_path = os.path.join(DATA_ROOT_PATH, filename)

sheet_name = ["Chemical Annotation", "Sample Meta Data", "Log Transformed Data"]

data = pd.read_excel(file_path, engine='openpyxl', sheet_name=sheet_name[0])

# Selecting only rows whose TYPE is NAMED
data = data[data["TYPE"] == "NAMED"]

# Selecting rows with INCHIKEY
data_INCHI = data.dropna(subset=["INCHIKEY"])
data_INCHI["INCHIKEY"] = "inchikey:"+data_INCHI["INCHIKEY"]

# Selecting rows without INCHIKEY
data_ = data[data['INCHIKEY'].isna()]

# From the without INCHI dataframe, select rows with KEGG id
data_ = data_.dropna(subset=["KEGG"])
data_["KEGG"] = data_["KEGG"].apply(lambda x:x.split(","))
data_ = data_.explode("KEGG")
data_["KEGG"] = "kegg.compound:" + data_["KEGG"]



## Finding INCHIKEY for those compounds with only KEGG ids in iMSMS data

In [4]:
auth = basic_auth(SPOKE_USER, SPOKE_PASSWORD)
sdb = GraphDatabase.driver(URI, auth=auth)
query = "MATCH(n:Compound) WHERE '{}' IN n.xrefs RETURN n.identifier AS n_id"
sdb = GraphDatabase.driver(URI, auth=auth)
node_list = []
with sdb.session() as session:
    with session.begin_transaction() as tx:
        for index,row in data_.iterrows():
            result = tx.run(query.format(row["KEGG"]))
            for row_ in result:
                node_list.append((row["KEGG"], row_["n_id"]))
sdb.close()
node_df = pd.DataFrame(node_list, columns=["KEGG", "identifer"])
node_df
        


,KEGG,identifer
0,kegg.compound:C02341,inchikey:GTZCVFVGUGFEME-HNQUOIGGSA-N
1,kegg.compound:C00803,inchikey:NQPDZGIKBAWPEJ-UHFFFAOYSA-N


In [5]:
data_KEGG_merge = pd.merge(data_, node_df, on="KEGG")


## Selecting only relevant columns for mapping

In [6]:
data_INCHI_map = data_INCHI[["CHEM_ID", "CHEMICAL_NAME", "INCHIKEY"]] 
data_KEGG_merge_map = data_KEGG_merge[["CHEM_ID", "CHEMICAL_NAME", "identifer"]] 
data_KEGG_merge_map = data_KEGG_merge_map.rename(columns={"identifer":"INCHIKEY"})
data_final_map = pd.concat([data_INCHI_map, data_KEGG_merge_map], ignore_index=True)
data_final_map = data_final_map.rename(columns={"CHEMICAL_NAME": "name", "INCHIKEY": "spoke_identifer"})
data_final_map

,CHEM_ID,name,spoke_identifer
0,35,S-1-pyrroline-5-carboxylate,inchikey:DWAKNKKXGALPNW-UHFFFAOYSA-N
1,50,spermidine,inchikey:ATHGHQPFGPMSJY-UHFFFAOYSA-N
2,55,1-methylnicotinamide,inchikey:LDHMAVIPBRSVRG-UHFFFAOYSA-N
3,62,"12,13-DiHOME",inchikey:CQSLTKIXAJTQGA-FLIBITNWSA-N
4,93,alpha-ketoglutarate,inchikey:KPGXRSRHYNQIFN-UHFFFAOYSA-N
...,...,...,...
1201,100022127,tetrahydrocortisone glucuronide (5),inchikey:QUOCEDQXFGCYTL-QMELEVSMSA-N
1202,100022172,perfluorohexanesulfonate (PFHxS),inchikey:QZHDEAJFRJCDMF-UHFFFAOYSA-N
1203,100022475,menthol glucuronide,inchikey:CLJGMBYGTHRUNF-PJQJKGEDSA-N
1204,100001359,aconitate [cis or trans],inchikey:GTZCVFVGUGFEME-HNQUOIGGSA-N


## Saving the mapping files

In [ ]:
short_chain_spoke_map_df.to_csv(os.path.join(OUTPUT_PATH, "short_chain_fatty_acid_spoke_map.csv"), index=False, header=True)
data_final_map.to_csv(os.path.join(OUTPUT_PATH, "global_metabolomics_compound_spoke_map.csv"), index=False, header=True)


## Checking iMSMS global compounds whose inchikeys do not match with any SPOKE nodes

In [56]:
iMSMS_compounds_not_in_SPOKE_df = pd.read_csv(os.path.join(OUTPUT_PATH, "iMSMS_compounds_not_in_SPOKE.csv"))
iMSMS_compounds_not_in_SPOKE_with_names_df = data_final_map[data_final_map.spoke_identifer.isin(iMSMS_compounds_not_in_SPOKE_df.CompoundIdentifier)]
print("There are {} iMSMS global compounds whose inchikeys do not map to any SPOKE nodes".format(iMSMS_compounds_not_in_SPOKE_with_names_df.CHEM_ID.unique().shape[0]))


There are 439 iMSMS global compounds whose inchikeys do not map to any SPOKE nodes


## Mapping to SPOKE based on xrefs

In [ ]:
%%time

auth = basic_auth(SPOKE_USER, SPOKE_PASSWORD)
sdb = GraphDatabase.driver(URI, auth=auth)

query = """
            MATCH (n:Compound)
            WHERE ANY(xref IN n.xrefs WHERE xref IN {})
            RETURN n.identifier as n_id;
        """

iMSMS_compounds_not_in_SPOKE_with_names_xref_df = data_INCHI[data_INCHI.INCHIKEY.isin(iMSMS_compounds_not_in_SPOKE_with_names_df.spoke_identifer)]
iMSMS_compounds_not_in_SPOKE_mapped_to_SPOKE = []
iMSMS_compounds_not_in_SPOKE_with_names_xref_df["KEGG"] = "kegg.compound:"+iMSMS_compounds_not_in_SPOKE_with_names_xref_df["KEGG"]
iMSMS_compounds_not_in_SPOKE_with_names_xref_df["PUBCHEM"] = "pubchem.compound:"+iMSMS_compounds_not_in_SPOKE_with_names_xref_df["PUBCHEM"]
with sdb.session() as session:
    with session.begin_transaction() as tx:
        for index, row in iMSMS_compounds_not_in_SPOKE_with_names_xref_df.iterrows():
            id_list = list(row[["KEGG", "PUBCHEM"]])
            result = tx.run(query.format(id_list))
            for row_ in result:
                iMSMS_compounds_not_in_SPOKE_mapped_to_SPOKE.append((row["CHEM_ID"], row["CHEMICAL_NAME"], row["INCHIKEY"], row_["n_id"]))

sdb.close()
iMSMS_compounds_not_in_SPOKE_mapped_to_SPOKE_df = pd.DataFrame(iMSMS_compounds_not_in_SPOKE_mapped_to_SPOKE, columns=["CHEM_ID", "name", "spoke_identifer", "spoke_identifer_newly_mapped"])

iMSMS_compounds_not_in_SPOKE_mapped_to_SPOKE_df.to_csv(os.path.join(OUTPUT_PATH, "iMSMS_compounds_KEGG_PUBCHEM_mapped_to_SPOKE.csv"), index=False, header=True)


## Mapping to SPOKE based on synonyms

#### Note: We are considering the mapping based on xrefs, since that is an ID based mapping

In [14]:
%%time

auth = basic_auth(SPOKE_USER, SPOKE_PASSWORD)
sdb = GraphDatabase.driver(URI, auth=auth)

query = """
            MATCH (n:Compound)
            WHERE ANY(synonym IN n.synonyms WHERE toLower(synonym) = toLower("{}"))
            RETURN n.identifier as n_id
        """

iMSMS_compounds_not_in_SPOKE_mapped_to_SPOKE = []
with sdb.session() as session:
    with session.begin_transaction() as tx:
        for index, row in iMSMS_compounds_not_in_SPOKE_with_names_df.iterrows():
            result = tx.run(query.format(row["name"]))
            for row_ in result:
                iMSMS_compounds_not_in_SPOKE_mapped_to_SPOKE.append((row["CHEM_ID"], row["name"], row["spoke_identifer"], row_["n_id"]))

sdb.close()
iMSMS_compounds_not_in_SPOKE_mapped_to_SPOKE_df = pd.DataFrame(iMSMS_compounds_not_in_SPOKE_mapped_to_SPOKE, columns=["CHEM_ID", "name", "spoke_identifer", "spoke_identifer_newly_mapped"])



CPU times: user 735 ms, sys: 63.8 ms, total: 798 ms
Wall time: 49min 54s


## Amending the initial global_metabolomics_compound_spoke_map 

In [84]:
data_final_map_inchimap_group = data_final_map[~data_final_map.spoke_identifer.isin(iMSMS_compounds_not_in_SPOKE_mapped_to_SPOKE_df.spoke_identifer)]
iMSMS_compounds_not_in_SPOKE_mapped_to_SPOKE_df_ = iMSMS_compounds_not_in_SPOKE_mapped_to_SPOKE_df.drop("spoke_identifer", axis=1)
iMSMS_compounds_not_in_SPOKE_mapped_to_SPOKE_df_ = iMSMS_compounds_not_in_SPOKE_mapped_to_SPOKE_df_.rename(columns={"spoke_identifer_newly_mapped":"spoke_identifer"})
data_final_map_ = pd.concat([data_final_map_inchimap_group, iMSMS_compounds_not_in_SPOKE_mapped_to_SPOKE_df_], ignore_index=True)
data_final_map_.to_csv(os.path.join(OUTPUT_PATH, "global_metabolomics_compound_spoke_map.csv"), index=False, header=True)
